 # Heroes of Pymoli Trend Analysis

+ Majority of players are male 84% who are generating \$1,967
+ Female players are on average more likely to purchase more expensive items and bring more value per purchase. 
+ About 44% of the players are in age group between 20 - 24
+ With 179 unique items, there was 780 purchases that generated \$2,379


In [59]:
###Importing Dependencies 
import pandas as pd
import numpy as np

### File to Load (Remember to Change These)
file_to_load = "Resources/purchase_data.csv"

### Read Purchasing File and store into Pandas data frame
df = pd.read_csv(file_to_load)


In [60]:
# Reading the dataframe
df.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


In [61]:
### Explore columns
df.columns

Index(['Purchase ID', 'SN', 'Age', 'Gender', 'Item ID', 'Item Name', 'Price'], dtype='object')

In [62]:
# Display the total number of players
total_players = df.loc[:,["Gender", "SN", "Age"]]
total_players = total_players.drop_duplicates()
players = total_players.count()[0]
totals = pd.DataFrame({"Total Players" :[players]})
totals

,Total Players
0,576


In [63]:
# Basic Purchasing Summary ( created by using dictionary and list)
summary = pd.DataFrame({"Unique Items": [len(df["Item Name"].unique())], 
                        "Average Price": [round(df["Price"].mean(),2)],
                        "Number of Purchased": [len(df["Purchase ID"].value_counts())],
                        "Total Revenue": [df["Price"].sum()]})

# Applying format to two columns to include $ signs and correct decimal place
summary["Average Price"] = summary["Average Price"].map("${:,.2f}".format)
summary["Total Revenue"] = summary["Total Revenue"].map("${:,.2f}".format)

# Displaying the summary
summary.head()

,Unique Items,Average Price,Number of Purchased,Total Revenue
0,179,$3.05,780,"$2,379.77"


In [64]:
# General Demographics Analysis

total_by_gender = df.groupby("Gender")["SN"].nunique()
percent_total = round(total_by_gender/576*100,2).map("%{:,.2f}".format)

#summary on general demographics 
gd_summary = pd.DataFrame({"Total Count": total_by_gender,"Percentage of Players":percent_total})
gd_summary.sort_values(by=['Total Count'],ascending=False)


,Total Count,Percentage of Players
Gender,,
Male,484,%84.03
Female,81,%14.06
Other / Non-Disclosed,11,%1.91


In [65]:
# Purchasing Analysis by Gender
purchase_by_gen = df.groupby(["Gender"])['Price'].count()
price_by_gen = df.groupby(["Gender"])['Price'].sum()
gender = df.groupby(["Gender"])
avg_by_gen = round(price_by_gen/purchase_by_gen,2).map("${:,.2f}".format)

# Remove Duplicates
deduped = df.drop_duplicates(['SN'], keep='last')
deduped_df = (deduped.groupby(['Gender']).size())

# Calculations
total_avg_person = round(price_by_gen/deduped_df,2).map("${:,.2f}".format)
price_by_gen = (price_by_gen).map("${:,.2f}".format)

# Purchase Summary
pur_sum = pd.DataFrame({"Purchase Count": purchase_by_gen, "Average Purchase Price": avg_by_gen, "Total Purchase Value": price_by_gen, "Avg Total Purchase per Person": total_avg_person})
pur_sum


,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Gender,,,,
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,"$1,967.64",$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


In [66]:
# Age Demographics
# Age Bins
age_bins = [0, 9.9, 14.9, 19.9, 24.9, 29.9, 34.9, 39.9, 100]

# Create labels for these bins
group_names = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]

# Addressing the duplicates for age_group
age_groups = df.loc[:,["Gender", "SN", "Age"]]
age_groups.drop_duplicates(['SN', 'Gender', "Age"], inplace=True, keep='first')

# Binning data
age_groups["Age Group"] = pd.cut(age_groups["Age"], age_bins, labels = group_names)

# Calculating and Count, Percentage of players and creating Summary
summary = pd.DataFrame()
summary['Total Count'] = age_groups.groupby(['Age Group'])['Age'].count()
summary['% of Players']=(age_groups.groupby(['Age Group'])['Age'].count() * 100)/574
summary['% of Players']=summary['% of Players'].map("%{:,.2f}".format)
summary

,Total Count,% of Players
Age Group,,
<10,17,%2.96
10-14,22,%3.83
15-19,107,%18.64
20-24,258,%44.95
25-29,77,%13.41
30-34,52,%9.06
35-39,31,%5.40
40+,12,%2.09


In [67]:
# PURCHASING ANALYSIS (AGE)
# Creating Bins

purch_df = df.copy()
purch_df["Age Group"] = pd.cut(purch_df["Age"],age_bins, labels = group_names)

# Calculating Purchase Count, Purchase Price, Purchase value and Average Purchase Total Price Per Person
purch_ct = purch_df.groupby(["Age Group"])["Price"].count()
purch_price = purch_df.groupby(["Age Group"])["Price"].mean().round(2)
purch_value = purch_df.groupby(["Age Group"])["Price"].sum().round(2)
avg_purch_total_pp = purch_value / summary["Total Count"].round(2)

# Creating Summary DataFrame
Age_Purchasing_Analysis = pd.DataFrame({"Purchase Count": purch_ct,
                                            "Average Purchase Price": purch_price,
                                            "Total Purchase Value": purch_value,
                                            "Avg Total Purchase Per Person": avg_purch_total_pp})
# Formating
Age_Purchasing_Analysis["Average Purchase Price"] = Age_Purchasing_Analysis["Average Purchase Price"].map("${:.2f}".format)
Age_Purchasing_Analysis["Total Purchase Value"] = Age_Purchasing_Analysis["Total Purchase Value"].map("${:.2f}".format)
Age_Purchasing_Analysis["Avg Total Purchase Per Person"] = Age_Purchasing_Analysis["Avg Total Purchase Per Person"].map("${:.2f}".format)

Age_Purchasing_Analysis

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase Per Person
Age Group,,,,
<10,23,$3.35,$77.13,$4.54
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,$1114.06,$4.32
25-29,101,$2.90,$293.00,$3.81
30-34,73,$2.93,$214.00,$4.12
35-39,41,$3.60,$147.67,$4.76
40+,13,$2.94,$38.24,$3.19


In [68]:
### TOP SPENDERS ###

spend_analysis = df.groupby("SN")

# Calculating Purchase Count, Average Spend Price and Total Spend Purchse
top_purchase_ct = spend_analysis["Purchase ID"].count().round(2)
avg_sp_price = spend_analysis["Price"].mean().round(2)
total_spend_purchase = spend_analysis["Price"].sum().round(2)

# Creating Spender Summary Dataframe
spender_summary = pd.DataFrame({"Purchase Count": top_purchase_ct,
                                    "Avg Purchase Price": avg_sp_price,
                                    "Total Purchase Value": total_spend_purchase})

# Sorting and pulling top 5 spenders
top_spender_summary = spender_summary.sort_values(["Total Purchase Value"], ascending=False).head(5)
# Format the curencies 
top_spender_summary["Avg Purchase Price"] = top_spender_summary["Avg Purchase Price"].map("${:.2f}".format)
top_spender_summary["Total Purchase Value"] = top_spender_summary["Total Purchase Value"].map("${:.2f}".format)

top_spender_summary

,Purchase Count,Avg Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


In [69]:
### Most Popular Items

popularity = df.groupby(["Item ID", "Item Name"])

# Calculating Purchse Count, Item Price and Total Purchse Value

purchase_ct = popularity["Price"].count()
purchase_value = popularity["Price"].sum()
total_purchase_value = purchase_value/purchase_ct

# Creating Data Frame Item Popularity
item_popularity = pd.DataFrame({"Purchase Count" : purchase_ct,
                                  "Item Price": total_purchase_value,
                                  "Total Purchase Value": purchase_value})

# Sorting values by Purchase Count and selecting top 5
top_popular_items = item_popularity.sort_values("Purchase Count", ascending=False).head(5)

# Addressing formating
top_popular_items["Item Price"] = top_popular_items["Item Price"].map("${:,.2f}".format)
top_popular_items["Total Purchase Value"] = top_popular_items["Total Purchase Value"].map("${:,.2f}".format)

top_popular_items

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77
82,Nirvana,9,$4.90,$44.10
19,"Pursuit, Cudgel of Necromancy",8,$1.02,$8.16


In [70]:
### Most Profitable Items

# Sorting the table
most_profitable_items = item_popularity.sort_values("Total Purchase Value", ascending=False)

# Format with currency style
most_profitable_items["Item Price"] = most_profitable_items["Item Price"].map("${:,.2f}".format)
most_profitable_items["Total Purchase Value"] = most_profitable_items["Total Purchase Value"].map("${:,.2f}".format)
# Displaying top 5 items. 
most_profitable_items.head(5)

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
92,Final Critic,8,$4.88,$39.04
103,Singed Scalpel,8,$4.35,$34.80
